In [3]:
%matplotlib inline
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string

In [4]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [5]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [6]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [7]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

NameError: name 'rmStr' is not defined

In [ ]:
# Merging the datasets

#merge_ans=answer_[['anon_id','course_id']] #sets answer df up for merging 
#student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student.merge(answer_,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs=student_ans.merge(course_, on='course_id')
#.drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [ ]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# Whole Dataset

In [ ]:
stu_ans_crs

In [ ]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id','question_id','text']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1

sac_lvls_rkc=sac_lvls_rkc.reset_index()

sac_lvls_rkc

In [ ]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('question_id','nunique'), lvl_nums=('question_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [ ]:
unique_q=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

In [ ]:
unique_q

In [ ]:
unique_t=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
unique_a=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()
unique_a

In [ ]:
sac_lvls_rkc=sac_lvls_rkc.reset_index()
sac_lvls_rkc

In [ ]:
#dividing my level 3 and language
sac_lvls_rkc3=sac_lvls_rkc

index=0
for i in sac_lvls_rkc3.level_id:
    if i!=3:
        sac_lvls_rkc3=sac_lvls_rkc3.drop(index)
    index+=1
    
sac_lvls_rkc3

In [ ]:
unique_q=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [ ]:
unique_q

In [ ]:
unique_t

In [ ]:
unique_a

In [ ]:
#dividing my level 4 and language
sac_lvls_rkc4=sac_lvls_rkc

index=0
for i in sac_lvls_rkc4.level_id:
    if i!=4:
        sac_lvls_rkc4=sac_lvls_rkc4.drop(index)
    index+=1
    
sac_lvls_rkc4=sac_lvls_rkc4.reset_index()

In [ ]:
unique_q=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [ ]:
unique_t

In [ ]:
unique_a

In [ ]:
#dividing my level 5 and language
sac_lvls_rkc5=sac_lvls_rkc

index=0
for i in sac_lvls_rkc5.level_id:
    if i!=5:
        sac_lvls_rkc5=sac_lvls_rkc5.drop(index)
    index+=1
    
sac_lvls_rkc5=sac_lvls_rkc5.reset_index()

In [ ]:
unique_q=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [ ]:
unique_t

In [ ]:
unique_a

# Loose Dataset

In [ ]:
loose=loose.dropna()
loose

In [ ]:
    # this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose

In [ ]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

all_three_loose.sort()
len(all_three_loose)

In [ ]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
in34_loose.sort()
print(len(in34_loose))

In [ ]:
# creates list of participants in loose sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
in45_loose.sort()
print(len(in45_loose))

# Finding number of texts breakdown

In [ ]:
all_three_loose

In [ ]:
all_three=loose
index=0
for i in all_three.anon_id:
    if i not in all_three_loose:
        all_three=all_three.drop(index)
    index+=1
    
all_three=all_three.reset_index()
all_three

In [ ]:
loose34=loose
index=0
for i in loose34.anon_id:
    if i not in in34_loose:
        loose34=loose34.drop(index)
    index+=1
    
loose34=loose34.reset_index()
loose34

In [ ]:
loose45=loose
index=0
for i in loose45.anon_id:
    if i not in in45_loose:
        loose45=loose45.drop(index)
    index+=1
    
loose45=loose45.reset_index()
loose45

In [ ]:
unique_t=all_three.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
unique_t=loose34.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
unique_t=loose45.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [24]:
#finds the number of texts per language in the given list of anon_ids
def find_counts(df,lang_col):
    spanish=0
    chinese=0
    korean=0
    index=0
    for i in df["answer_id"]:
        if df.at[index,lang_col]=='Spanish':
            spanish+=1
        elif df.at[index,lang_col]=='Chinese':
            chinese+=1
        elif df.at[index,lang_col]=='Korean':
            korean+=1
        index+=1
       # anon_id=str(df.at[index,'anon_id'])
    return(spanish,chinese,korean)

In [ ]:
#finds the token average of the given level in the given level group (id_list) for each language
def tokenAvg(df,textlencol,lang, id_list,lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #number of questions
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl and df.at[index,'anon_id'] in id_list:
            textlen=df.at[index,textlencol]
            textlen=int(textlen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                qcountc+=1
        index+=1
        
    print("Spanish Average: ",spanish/qcounts)
    print("Korean Average: ",korean/qcountk)
    print("Chinese Average: ",chinese/qcountc)

In [ ]:
#finding the token/word count ratio for each language
def ratio(df,textlencol,typelencol,lang, id_list,lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #unique token count
    tspanish=0
    tkorean=0
    tchinese=0
    #total question count
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl and df.at[index,'anon_id'] in id_list:
            textlen=df.at[index,textlencol]
            textlen=int(textlen)
            typelen=df.at[index,typelencol]
            typelen=int(typelen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                tspanish+=typelen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                tkorean+=typelen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                tchinese+=typelen
                qcountc+=1
        index+=1

    #finding ratios    
    spanishavg=spanish/qcounts
    tspanishavg=tspanish/qcounts
    koreanavg=korean/qcountk
    tkoreanavg=tkorean/qcountk
    chineseavg=chinese/qcountc
    tchineseavg=tchinese/qcountc
    
    print("Spanish Ratio: ",tspanishavg/spanishavg)
    print("Korean Ratio: ",tkoreanavg/koreanavg)
    print("Chinese Ratio: ",tchineseavg/chineseavg)

In [ ]:
# function that counts the amount of tokens per language FINDS AVERAGE OF ALL LVLS IN SPECIFIED LIST
def finding_tokens(df,id_list,lang_col,text_len_col,qcounts):
    spanish=0
    korean=0
    chinese=0
    index=0
    lspanish=0
    lchinese=0
    lkorean=0
    
    for i in df['anon_id']:
        if i in id_list:
            lang=df.at[index,lang_col]
            length=int(df.at[index,text_len_col])
            if lang=='Spanish':
                lspanish+=1
                spanish+=length
            elif lang=='Korean':
                lkorean+=1
                korean+=length
            elif lang=='Chinese':
                lchinese+=1
                chinese+=length
        index+=1
   # print('Spanish Token Average: ',spanish,'\nKorean Token Average: ',korean,
    #      '\nChinese Token Average:',chinese)
    print('Spanish Token Average: ',spanish/lspanish,'\nKorean Token Average: ',korean/lkorean,
          '\nChinese Token Average:',chinese/lchinese)

In [18]:
index=0
textcount_loose=loose.copy()

textcount_loose['processed_text']=pd.NaT
textcount_loose['wordtype_len']=pd.NaT
words=[]
for text in textcount_loose['text3 (edits made to fix word counts)']:
    uwords=[]
    text=str(text).replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    textcount_loose.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    textcount_loose.at[index,'text3_len']=len(words)
    textcount_loose.at[index,'wordtype_len']=len(uwords)
    index+=1

In [20]:
textcount_loose.dropna(inplace=True)

In [21]:
textcount_loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,141.0,aj8,Chinese,5.0,17.0,114.0,"Today, I am going to describe one of my classm...",today i am going to describe one of my classma...,74
1,143.0,az8,Chinese,5.0,17.0,95.0,My niece is 3 years old who is my younger brot...,my niece is 3 years old who is my younger brot...,67
2,133.0,az2,Korean,5.0,17.0,128.0,"When I was in Germany, I met a friend who was ...",when i was in germany i met a friend who was f...,72
3,135.0,at8,Korean,5.0,17.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is anonname0 she is a my eli friend ...,60
4,188.0,eh9,Korean,5.0,17.0,97.0,Younghun is my best friend. His facial appeara...,younghun is my best friend his facial appearan...,70
...,...,...,...,...,...,...,...,...,...
1805,47845.0,ec1,Korean,4.0,6070.0,138.0,My best friend is Sohyun who graduated from th...,my best friend is sohyun who graduated from th...,80
1806,47882.0,cq1,Spanish,4.0,6070.0,80.0,"My soul mate is Melissa, who has been my frien...",my soul mate is melissa who has been my friend...,57
1807,48246.0,bv8,Chinese,4.0,6105.0,131.0,The quote that talks about a true friend is so...,the quote that talks about a true friend is so...,73
1808,48254.0,dr8,Korean,4.0,6105.0,162.0,I think this means that home is not just a pla...,i think this means that home is not just a pla...,79


In [30]:
loose3=textcount_loose.query('level_id==3').reset_index()
find_counts(loose3,'L1')

(60, 52, 98)

In [29]:
loose3

,index,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,62,2579.0,da6,Spanish,3.0,302.0,204.0,When I was leaving to the English class in my ...,when i was leaving to the english class in my ...,62
1,63,2570.0,fo8,Chinese,3.0,302.0,60.0,I went to declare to pick up my child yesterda...,i went to declare to pick up my child yesterda...,37
2,64,2593.0,gv1,Korean,3.0,302.0,72.0,"While I am going to home, I listened music. Wh...",while i am going to home i listened music when...,47
3,65,2600.0,aa9,Korean,3.0,302.0,60.0,"While I was taking the trolley yesterday, I sa...",while i was taking the trolley yesterday i saw...,37
4,66,2697.0,fv7,Korean,3.0,302.0,108.0,I had a reunion with my friens. They were taki...,i had a reunion with my friens they were takin...,70
...,...,...,...,...,...,...,...,...,...,...
205,1750,43550.0,cf6,Korean,3.0,5533.0,82.0,There are many reasons to travel. First a fami...,there are many reasons to travel first a famil...,55
206,1751,43568.0,fb9,Korean,3.0,5533.0,60.0,Vacation has many beautiful places. They have ...,vacation has many beautiful places they have m...,40
207,1752,43629.0,cf1,Korean,3.0,5533.0,105.0,Here are some suggestions about becoming healt...,here are some suggestions about becoming healt...,76
208,1753,43659.0,bm0,Korean,3.0,5533.0,97.0,A life in a small town has some disadvantages....,a life in a small town has some disadvantages ...,58
